In [1]:
import os
import yaml
import time
import torch
import epynet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from utils.epanet_loader import get_nx_graph
from utils.epanet_simulator import epanetSimulator
from utils.data_loader import battledimLoader, dataCleaner, dataGenerator, embedSignalOnGraph, rescaleSignal
from modules.torch_gnn import ChebNet
from utils.visualisation import visualise

In [2]:
def predict_pressure(graph, pressure_series, print_out_rate=100, save=True, filename='predictions.csv'):
    
    results = []
    elapsed_time = time.time()
    
    for i, partial_graph_signal in enumerate(pressure_series):
        if not i % print_out_rate:
            execution_time = time.time()
        
        results.append(model.predict(graph, partial_graph_signal))
        
        if not i % print_out_rate:
            print('Signal:\t{}\t Execution:\t{:.3f} s\t Elapsed:\t{:.3f} s'.format(i,
                                                                                   time.time()-execution_time, 
                                                                                   time.time()-elapsed_time))
    if save:
        print('-'*63+"\nSaving results to: \n{}/{}\n\n".format(os.getcwd(),filename))
        pd.DataFrame(results).to_csv(filename)
    
    return results
    

In [3]:
# Runtime configuration
path_to_wdn     = './data/L-TOWN.inp'
path_to_data    = './data/l-town-data/'
weight_mode     = 'pipe_length'
self_loops      = True
scaling         = 'minmax'
task            = 'prediction'
mode            = 'n_timesteps'
n_timesteps     = 3
figsize         = (50,16)
print_out_rate  = 100               
model_name      = 'l-town-chebnet-' + weight_mode +'-' + scaling + '{}'.format('-self_loop' if self_loops else '')
last_model_path = './studies/models/l-town-chebnet-predictor-pipe_length-minmax-self_loop--31.pt'
last_log_path   = './studies/logs/l-town-chebnet-predictor-pipe_length-minmax-self_loop--31.csv' 

In [4]:
print('1. Making graph from EPANET input file \n')
    
# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
G , pos , head = get_nx_graph(wdn, weight_mode=weight_mode, get_head=True)

# Open the dataset configuration file
with open(path_to_data + 'dataset_configuration.yaml') as file:

    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader) 

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

if self_loops:
    for sensor_node in sensors:             # For each node in the sensor list
        G.add_edge(u_of_edge=sensor_node,   # Add an edge from that node ...
                    v_of_edge=sensor_node,   # ... to itself ...
                    weight=1.,name='SELF')   # ... and set its weight to equal 1

1. Making graph from EPANET input file 



In [5]:
print('2. Retrieving simulation data \n')

# Instantiate the nominal WDN model
nominal_wdn_model = epanetSimulator(path_to_wdn, path_to_data)

# Run a simulation
nominal_wdn_model.simulate()

# Retrieve the nodal pressures
nominal_pressure = nominal_wdn_model.get_simulated_pressure()

x,y,scale,bias = dataCleaner(pressure_df    = nominal_pressure, # Pass the nodal pressures
                                observed_nodes = sensors,          # Indicate which nodes have sensors
                                rescale        = scaling,          # Perform scaling on the timeseries data
                                mode           = mode,             # Set data curation mode (n_timesteps / sensor_mask)
                                task           = task,             # Set data curation task (prediction  / reconstruction)
                                n_timesteps    = n_timesteps)      # Perform scaling on the timeseries data

# Split the data into training and validation sets
x_trn, x_val, y_trn, y_val = train_test_split(x, y, 
                                                test_size    = 0.2,
                                                random_state = 1,
                                                shuffle      = False)

2. Retrieving simulation data 



In [6]:
print('3. Retrieving historical data \n')

# Load the data into a numpy array with format matching the GraphConvWat problem
pressure_2018 = battledimLoader(observed_nodes = sensors,
                                n_nodes        = 782,
                                path           = path_to_data,
                                file           = '2018_SCADA_Pressures.csv',
                                rescale        = True, 
                                scale          = scale,
                                bias           = bias,
                                task           = 'prediction',
                                mode           = 'n_timesteps',
                                n_timesteps    = 3)

pressure_2019 = battledimLoader(observed_nodes = sensors,
                                n_nodes        = 782,
                                path           = path_to_data,
                                file           = '2019_SCADA_Pressures.csv',
                                rescale        = True, 
                                scale          = scale,
                                bias           = bias,
                                task           = 'prediction',
                                mode           = 'n_timesteps',
                                n_timesteps    = 3)

3. Retrieving historical data 



In [13]:
pressure_2018.shape

(105117, 782, 3)

In [7]:
print('4. Loading trained GNN model \n')

# Set the computation device as NVIDIA GPU if available else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate a Chebysev Network GNN model
model  = ChebNet(name           = 'ChebNet',
                    data_generator = None,
                    device         = device, 
                    in_channels    = np.shape(x_trn)[-1], 
                    out_channels   = np.shape(y_trn)[-1],
                    data_scale     = scale, 
                    data_bias      = bias).to(device)

4. Loading trained GNN model 



In [9]:
last_model_path
last_log_path

'./studies/logs/l-town-chebnet-predictor-pipe_length-minmax-self_loop--31.csv'

In [8]:
# We offer the user the option to load the previously trained weights
model.load_model(last_model_path, last_log_path)


                Loaded previous model results...
                --------------------------------------------------
                Model has been trained for:	100 epochs
                Best validation loss:      	2.7710501698200315e-05 
                Occurred in training round:	37 


C:\Users\30806\OneDrive\UCL\project\Project\Training model\modules\torch_gnn.py:302: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path_to_mo

In [12]:
test = pressure_2018[:10, :, :]

In [20]:
test.shape

(10, 782, 3)

In [10]:
print('5.1 Predicting 2018 pressure\n')

prediction_2018 = predict_pressure(G, 
                                    pressure_2018,
                                    print_out_rate = print_out_rate, 
                                    save           = True, 
                                    filename       = "./studies/results/pred_pressure_2018.csv")

5.1 Predicting 2018 pressure

Signal:	0	 Execution:	2.531 s	 Elapsed:	2.531 s
Signal:	100	 Execution:	0.133 s	 Elapsed:	16.864 s
Signal:	200	 Execution:	0.131 s	 Elapsed:	30.644 s
Signal:	300	 Execution:	0.134 s	 Elapsed:	44.513 s
Signal:	400	 Execution:	0.135 s	 Elapsed:	58.114 s
Signal:	500	 Execution:	0.135 s	 Elapsed:	71.958 s
Signal:	600	 Execution:	0.145 s	 Elapsed:	86.152 s
Signal:	700	 Execution:	0.148 s	 Elapsed:	101.607 s
Signal:	800	 Execution:	0.150 s	 Elapsed:	117.240 s
Signal:	900	 Execution:	0.154 s	 Elapsed:	132.530 s
Signal:	1000	 Execution:	0.154 s	 Elapsed:	147.921 s
Signal:	1100	 Execution:	0.143 s	 Elapsed:	163.501 s
Signal:	1200	 Execution:	0.145 s	 Elapsed:	179.052 s
Signal:	1300	 Execution:	0.146 s	 Elapsed:	194.748 s
Signal:	1400	 Execution:	0.153 s	 Elapsed:	210.188 s
Signal:	1500	 Execution:	0.157 s	 Elapsed:	226.046 s
Signal:	1600	 Execution:	0.144 s	 Elapsed:	241.601 s
Signal:	1700	 Execution:	0.161 s	 Elapsed:	257.237 s
Signal:	1800	 Execution:	0.152 s	 El

In [11]:
print('5.2 Predicting 2019 pressure\n')
    
prediction_2019 = predict_pressure(G, 
                                    pressure_2019,
                                    print_out_rate = print_out_rate, 
                                    save           = True, 
                                    filename       = "./studies/results/pred_pressure_2019.csv")

5.2 Predicting 2019 pressure

Signal:	0	 Execution:	0.224 s	 Elapsed:	0.224 s
Signal:	100	 Execution:	0.155 s	 Elapsed:	15.095 s
Signal:	200	 Execution:	0.140 s	 Elapsed:	29.218 s
Signal:	300	 Execution:	0.136 s	 Elapsed:	43.448 s
Signal:	400	 Execution:	0.378 s	 Elapsed:	57.388 s
Signal:	500	 Execution:	0.134 s	 Elapsed:	71.112 s
Signal:	600	 Execution:	0.135 s	 Elapsed:	85.145 s
Signal:	700	 Execution:	0.136 s	 Elapsed:	99.051 s
Signal:	800	 Execution:	0.135 s	 Elapsed:	113.358 s
Signal:	900	 Execution:	0.138 s	 Elapsed:	127.502 s
Signal:	1000	 Execution:	0.139 s	 Elapsed:	141.338 s
Signal:	1100	 Execution:	0.137 s	 Elapsed:	155.355 s
Signal:	1200	 Execution:	0.137 s	 Elapsed:	169.443 s
Signal:	1300	 Execution:	0.142 s	 Elapsed:	183.356 s
Signal:	1400	 Execution:	0.133 s	 Elapsed:	197.706 s
Signal:	1500	 Execution:	0.138 s	 Elapsed:	212.011 s
Signal:	1600	 Execution:	0.136 s	 Elapsed:	225.933 s
Signal:	1700	 Execution:	0.137 s	 Elapsed:	240.063 s
Signal:	1800	 Execution:	0.136 s	 Ela